In [ ]:
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import nltk
import string
from os import getcwd
# from TurkishStemmer import TurkishStemmer
from tokenizers import Tokenizer
from trtokenizer.tr_tokenizer import SentenceTokenizer, WordTokenizer
from snowballstemmer import TurkishStemmer
from sklearn.model_selection import train_test_split

In [2]:
# get the data
df = pd.read_csv('input.csv', sep=';')
df

,speed,service,flavour,review
0,-,1,1,Her zaman komşu fırından sipariş verdiğim için...
1,-,10,2,sosisli ürün isteyen adama peynirli bişey yol...
2,-,10,10,Siparisimi cok hizli getiren ekip arkadasiniza...
3,1,1,7,"After waiting more tjan one hour, they didnt d..."
4,4,4,1,Iyi pişsin diye söylememe rağmen az pişmiş gel...
...,...,...,...,...
60237,10,10,10,Super.
60238,10,10,10,Mükemmelsiniz.
60239,10,10,10,Çorbası efsane mutlaka deneyin.
60240,10,10,10,Harikasınız!


In [3]:
df = df[df['review'].isna() == False]
df

,speed,service,flavour,review
0,-,1,1,Her zaman komşu fırından sipariş verdiğim için...
1,-,10,2,sosisli ürün isteyen adama peynirli bişey yol...
2,-,10,10,Siparisimi cok hizli getiren ekip arkadasiniza...
3,1,1,7,"After waiting more tjan one hour, they didnt d..."
4,4,4,1,Iyi pişsin diye söylememe rağmen az pişmiş gel...
...,...,...,...,...
60237,10,10,10,Super.
60238,10,10,10,Mükemmelsiniz.
60239,10,10,10,Çorbası efsane mutlaka deneyin.
60240,10,10,10,Harikasınız!


In [4]:
partially_filled_reviews = df[['speed', 'service', 'flavour']].apply(lambda row: '-' in row.values, axis=1)
partially_filled_reviews.sum()

df = df[~partially_filled_reviews]

In [5]:
df['review'] = df['review'].str.lower()
df

C:\Users\caner\AppData\Local\Temp\ipykernel_14948\2891927258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].str.lower()


,speed,service,flavour,review
3,1,1,7,"after waiting more tjan one hour, they didnt d..."
4,4,4,1,iyi pişsin diye söylememe rağmen az pişmiş gel...
5,1,1,1,kokmuş hamburger getirdiniz be ayıp ulan resm...
6,10,10,10,her yiyeceği özenle getiriyolar ve lezzeti old...
7,1,1,1,allah affetsin çok kötüydü hiç bir şey mi iyi ...
...,...,...,...,...
60237,10,10,10,super.
60238,10,10,10,mükemmelsiniz.
60239,10,10,10,çorbası efsane mutlaka deneyin.
60240,10,10,10,harikasınız!


In [6]:
df[['speed', 'service', 'flavour']] = df[['speed', 'service', 'flavour']].astype(int)

C:\Users\caner\AppData\Local\Temp\ipykernel_14948\219824738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['speed', 'service', 'flavour']] = df[['speed', 'service', 'flavour']].astype(int)


In [7]:
df.dtypes

speed       int64
service     int64
flavour     int64
review     object
dtype: object

In [8]:
df['label'] = np.where((df['service'] + df['speed'] + df['flavour']) / 3 > 5.0, 1, 0)
df

C:\Users\caner\AppData\Local\Temp\ipykernel_14948\3236536471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = np.where((df['service'] + df['speed'] + df['flavour']) / 3 > 5.0, 1, 0)


,speed,service,flavour,review,label
3,1,1,7,"after waiting more tjan one hour, they didnt d...",0
4,4,4,1,iyi pişsin diye söylememe rağmen az pişmiş gel...,0
5,1,1,1,kokmuş hamburger getirdiniz be ayıp ulan resm...,0
6,10,10,10,her yiyeceği özenle getiriyolar ve lezzeti old...,1
7,1,1,1,allah affetsin çok kötüydü hiç bir şey mi iyi ...,0
...,...,...,...,...,...
60237,10,10,10,super.,1
60238,10,10,10,mükemmelsiniz.,1
60239,10,10,10,çorbası efsane mutlaka deneyin.,1
60240,10,10,10,harikasınız!,1


In [94]:
df[df['review'] == "soğuk ve lezzetsiz."]

,speed,service,flavour,review,label
8535,1,1,1,soğuk ve lezzetsiz.,0
21267,1,1,1,soğuk ve lezzetsiz.,0
24012,1,1,1,soğuk ve lezzetsiz.,0
28684,2,2,1,soğuk ve lezzetsiz.,0


In [9]:
data = df[['review', 'label']]
data

,review,label
3,"after waiting more tjan one hour, they didnt d...",0
4,iyi pişsin diye söylememe rağmen az pişmiş gel...,0
5,kokmuş hamburger getirdiniz be ayıp ulan resm...,0
6,her yiyeceği özenle getiriyolar ve lezzeti old...,1
7,allah affetsin çok kötüydü hiç bir şey mi iyi ...,0
...,...,...
60237,super.,1
60238,mükemmelsiniz.,1
60239,çorbası efsane mutlaka deneyin.,1
60240,harikasınız!,1


In [ ]:
# get the positive and negative comments
positive_comments = data[data['label'] == 1]
negative_comments = data[data['label'] == 0]

# get the training set and test set
train_pos, test_pos = train_test_split(positive_comments, test_size=0.3, random_state=42)
train_neg, test_neg = train_test_split(negative_comments, test_size=0.3, random_state=31)

train_x = pd.concat([train_pos, train_neg], ignore_index=True)
test_x = pd.concat([test_pos, test_neg], ignore_index=True)

# shuffle the data
train_x = train_x.sample(frac=1, random_state=42).reset_index(drop=True)
test_x = test_x.sample(frac=1, random_state=42).reset_index(drop=True)

train_y = train_x['label'].values.tolist()
test_y = test_x['label'].values.tolist()

train_x = train_x.drop(columns=['label']).review.values.tolist()
test_x = test_x.drop(columns=['label']).review.values.tolist()

In [22]:
def process_comment(comment):
    stemmer = TurkishStemmer()
    stopwords_turkish = stopwords.words('turkish')
    tokenizer = nltk.WordPunctTokenizer()

    # TODO: handle miswritten words
    # detector = detector.TurkishNLP()
    # lwords = detector.list_words("vri kümsi idrae edre ancaka daha güezl oalbilir")
    # corrected_words = detector.auto_correct(lwords)
    # corrected_string = " ".join(corrected_words)

    # tokenize
    tokens = tokenizer.tokenize(comment)
    
    comment_clean = []
    for word in tokens:
        if (word not in string.punctuation and
            word not in stopwords_turkish):
            stem_word = stemmer.stemWord(word)
            comment_clean.append(stem_word)

    return comment_clean

In [18]:
# test it
comment = """ya aslinda yemek hizli geldi de ama lezzet ve servis icin  vasat bile denemezdi ya neyse. yorumuma da bakmamislardi ayrica.
bu da yeni paragraf baslangici idir"""
print(process_comment(comment))

['asl', 'yemek', 'hizli', 'gel', 'lezzet', 'servis', 'iç', 'vasat', 'bil', 'denemez', 'ne', 'yorum', 'bakmamis', 'ayriç', 'ye', 'paragraf', 'baslangiç', 'i']


In [23]:
def count_comments(comments, ys):
    freqs = {}
    for comment, y in zip(comments, ys):
        for word in process_comment(comment):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs


In [93]:
# Testing your function
comments = ['fena degildi', 'guzeldi bayagi sadece biraz fazla baharatliydi o azaltilabilir', 'hem yavas hem eksik geldi begenmedim maalesef fena kotuydu', 'kesinlike oneririm']
ys = [1, 1, 0, 1]
count_comments(comments, ys)

{('fena', 1): 1,
 ('degil', 1): 1,
 ('guzel', 1): 1,
 ('bayagi', 1): 1,
 ('sadeç', 1): 1,
 ('biraz', 1): 1,
 ('faz', 1): 1,
 ('baharatli', 1): 1,
 ('azaltilabilir', 1): 1,
 ('yavas', 0): 1,
 ('eksik', 0): 1,
 ('gel', 0): 1,
 ('begenmedi', 0): 1,
 ('maalesef', 0): 1,
 ('fena', 0): 1,
 ('kot', 0): 1,
 ('kesinlik', 1): 1,
 ('onerir', 1): 1}

In [24]:
freqs = count_comments(train_x, train_y)

In [ ]:
def train_naive_bayes(freqs, train_y):
    loglikelihood = {}

    # calculate unique words in the freqs dictionary
    vocab = set([pair[0] for pair in freqs.keys()])
    v = len(vocab)

    # calculate the  total positive words and negative words
    n_pos = n_neg = 0
    for pair in freqs.keys():
        if pair[1] == 1:
            n_pos += freqs[pair]
        else:
            n_neg += freqs[pair]
    
    # calculate D, D_pos, and D_neg
    d = len(train_y)        # all comments
    d_pos = sum(train_y)    # positive comments
    d_neg = d - d_pos       # negative comments
    
    # calculate logprior
    logprior = np.log(d_pos) - np.log(d_neg)

    for word in vocab:
        # calculate freq_pos and freq_neg
        freq_pos = freqs[(word, 1)] if (word, 1) in freqs.keys() else 0
        freq_neg = freqs[(word, 0)] if (word, 0) in freqs.keys() else 0

        # calculate the probability of each word
        p_w_pos = (freq_pos + 1) / (n_pos + v)
        p_w_neg = (freq_neg + 1) / (n_neg + v)

        loglikelihood[word] = np.log(p_w_pos / p_w_neg)
    
    return logprior, loglikelihood

In [ ]:
logprior, loglikelihood = train_naive_bayes(freqs, train_y)
print(logprior)
print(len(loglikelihood))

1.8417232237064045
21342


In [33]:
def naive_bayes_predict(comment, logprior, loglikelihood):
    # get the words
    words = process_comment(comment)

    # calculate the probability that the comment belongs to the positive or negative class
    prob = 0
    prob += logprior

    for word in words:
        if word in loglikelihood:
            prob += loglikelihood[word]

    return prob

In [51]:
comment = "asiri kotu igrenc rezalet asla :) :)"
print(naive_bayes_predict(comment, logprior, loglikelihood))

-0.11392790313313839


In [41]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    y_predicted = []

    for comment in test_x:
        if naive_bayes_predict(comment, logprior, loglikelihood) > 0:
            y_predicted.append(1)
        else:
            y_predicted.append(0)

    error = np.average(np.absolute(np.array(y_predicted) - np.array(test_y)))
    accuracy = 1 - error

    return accuracy

In [42]:
print(test_naive_bayes(test_x, test_y, logprior, loglikelihood))

0.9063936238291931


In [71]:
def get_ratio(freqs, word):
    pos_neg_ratio = {}

    pos_neg_ratio['positive'] = freqs[(word, 1)] if (word, 1) in freqs else 0
    pos_neg_ratio['negative'] = freqs[(word, 0)] if (word, 0) in freqs else 0
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1) / (pos_neg_ratio['negative'] + 1)

    return pos_neg_ratio

In [80]:
def get_words_by_threshold(freqs, label, threshold):
    words = {}

    for key in freqs.keys():
        word, _ = key
        pos_neg_ratio = get_ratio(freqs, word)
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:
            words[word] = pos_neg_ratio
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
            words[word] = pos_neg_ratio

    return words

In [ ]:
get_words_by_threshold(freqs, label=1, threshold=10)

In [ ]:
comment = 'hiz olarak gayet basarili ama lezzet olarak çok kötüydü'
print(naive_bayes_predict(comment, logprior, loglikelihood))

10.33024827435203


In [92]:
print('Truth - Predicted - Comment')
for x, y in zip(test_x, test_y):
    y_predicted = naive_bayes_predict(x, logprior, loglikelihood)

    if y != (np.sign(y_predicted) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_predicted) > 0, ' '.join(
            process_comment(x)).encode('ascii', 'ignore')))

Truth - Predicted - Comment
1	0.00	b'soguk lezzetsiz'
0	1.00	b'corba yogu degil ada basaril guzel servis yapa elema maskes yari be cok rahtsiz et ve par us veremedi boyle sey sogutyor'
0	1.00	b'siparis souk gel'
0	1.00	b'ncelik sipari 2 acl 2 acs lahma sipari ver ancak acsz nce sipari vermi b bir sor yaamad ayr i harc faz yal tuzlu'
0	1.00	b'beenmedi'
0	1.00	b'yemek lezzet yok'
0	1.00	b'ada kebap tad gzel deil bulgur ayn'
0	1.00	b'bayat'
1	0.00	b'sipari ketap mayonez pee slak mendil koyulmam zensiz'
1	0.00	b'tadi gzel ula 30 40 dk diyodu 15 dk gel malzemel ba a parmak u r kooymus'
0	1.00	b'hot dog inanlmaz kt'
0	1.00	b'dur az kk'
1	0.00	b'abartmyor hayat yedik kt ada dr us p ke yedi mi'
1	0.00	b'lahmacun acs istiyor acl geliyor stelik sipari kartrp gnderiyor 2 defa yanl ipari geliyor'
0	1.00	b'eksik gele sipari mehur'
0	1.00	b'restorant yetkilis ba ular anlatr gereke lezzet s sor yetkil bi ap ulam zaman ba ular sevinir ..'
1	0.00	b'acs lahma sipari et acl gel mide rahatsz yiyemedi'
1	0